## Imports and Setup

In [1]:
import os
import pandas as pd
import geopandas as gpd
import logging
import joblib
import torch

import sys
sys.path.insert(0, "../src")
import sat_download

sys.path.insert(0, "../utils/")
import data_utils
import config_utils
import pred_utils
import embed_utils

%load_ext autoreload
%autoreload 2

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


## Load Config Files

In [2]:
iso_code = "SEN"
adm_level = "ADM2"
shapename = "Gossas"
spacing = 150
buffer_size = 150

cwd = os.path.dirname(os.getcwd())
data_config_file = os.path.join(cwd, "configs/data_config.yaml")
data_config = config_utils.load_config(data_config_file)

sat_config_file = os.path.join(cwd, "configs/sat_configs/sat_config_500x500_60cm.yaml")
sat_creds_file = os.path.join(cwd, "configs/sat_configs/issa_sat_creds.yaml")

sat_config = config_utils.load_config(sat_config_file)
sat_creds = config_utils.create_config(sat_creds_file)

## Generate Prediction Tiles

In [3]:
tiles = pred_utils.generate_pred_tiles(
    data_config, iso_code, spacing, buffer_size, adm_level, shapename
).reset_index()
print(f"Total tiles: {tiles.shape}")
tiles.head(3)

100%|████████████████████| 36/36 [04:19<00:00,  7.20s/it]                                                               


Total tiles: (9162, 12)


,index,geometry,shapeName,shapeISO,shapeID,shapeGroup,shapeType,points,index_right,type,properties,UID
0,11358,"POLYGON ((-1800194.630 1621293.203, -1800194.6...",Gossas,,32543966B1237769986254,SEN,ADM2,POINT (-1800344.630 1621143.203),135242,Feature,"{'height': -1.0, 'confidence': -1.0}",11358
1,11359,"POLYGON ((-1800194.630 1621443.203, -1800194.6...",Gossas,,32543966B1237769986254,SEN,ADM2,POINT (-1800344.630 1621293.203),135242,Feature,"{'height': -1.0, 'confidence': -1.0}",11359
2,11656,"POLYGON ((-1800044.630 1621293.203, -1800044.6...",Gossas,,32543966B1237769986254,SEN,ADM2,POINT (-1800194.630 1621143.203),135242,Feature,"{'height': -1.0, 'confidence': -1.0}",11656


## Download Satellite Images

In [4]:
data = tiles.copy()
data["geometry"] = data["points"]
sat_dir = os.path.join(cwd, "output", iso_code, "images", shapename)
sat_download.download_sat_images(sat_creds, sat_config, data=data, out_dir=sat_dir)

INFO:root:Data dimensions: (9162, 14), CRS: EPSG:3857
100%|████████████████████| 9162/9162 [00:05<00:00, 1795.35it/s]                                                         


## CNN Model

In [5]:
model_config_file = os.path.join(cwd, f"configs/cnn_configs/vgg16.yaml")
model_config = config_utils.load_config(model_config_file)
result = pred_utils.cnn_predict(tiles, iso_code, shapename, model_config, sat_dir, n_classes=2)

INFO:root:Device: cpu
INFO:root:Model file /home/itingzon.unicef/giga/exp/SEN_vgg16/SEN_vgg16.pth successfully loaded.
100%|████████████████████| 9162/9162 [2:32:48<00:00,  1.00s/it]                                                         


## ViT Model

In [13]:
model_config_file = os.path.join(cwd, f"configs/model_configs/dinov2_vitl14-LR.yaml")
model_config = config_utils.load_config(model_config_file)
result = pred_utils.vit_pred(tiles, model_config, iso_code, shapename, sat_dir)

Using cache found in /home/itingzon.unicef/.cache/torch/hub/facebookresearch_dinov2_main
INFO:dinov2:using MLP layer as FFN
INFO:root:Device: cpu
INFO:root:Generating embeddings...
INFO:root:Reading file /home/itingzon.unicef/giga/output/SEN/embeddings/SEN_Foundiougne_dinov2_vitl14_embeds.csv
INFO:root:Loaded /home/itingzon.unicef/giga/exp/SEN-dinov2_vitl14-LR/SEN-dinov2_vitl14-LR.pkl
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex:sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:root:Generated /home/itingzon.unicef/giga/output/SEN/results/SEN_Foundiougne_dinov2_vitl14-LR_results.gpkg
